In [1]:
from accuracy_bounds.data.sentinel2 import SRDataset
from torch.utils.data import DataLoader
from accuracy_bounds.inverseproblems.utils import torch_csr_to_scipy, torch_sparse_to_scipy_csr
from accuracy_bounds.inverseproblems.kersize_compute_dataloader import (target_distances_samplingYX_perbatch_cuda,
    kersize_samplingYX,
    avgLB_samplingYX,
    avgkersize_samplingYX)

In [2]:
data_path = "/localhome/gawl_ja/Dokumente/Kernel/archive"
batch_size = 100

In [3]:
input_data = SRDataset(folder_path=data_path, suffixes=('lr_res'))
target_data1 = SRDataset(folder_path=data_path, suffixes=('hr_res'))
target_data2 = SRDataset(folder_path=data_path, suffixes=('hr_res'))
forwarded_target_data = SRDataset(folder_path=data_path, suffixes=('lr_res'))

print("Prepared Datasets")
print("    input_data:           ", len(input_data))
print("    target_data1:          ", len(target_data1))
print("    target_data:2          ", len(target_data2))
print("    forwarded_target_data:", len(forwarded_target_data))

Prepared Datasets
    input_data:            1999
    target_data1:           2000
    target_data:2           2000
    forwarded_target_data: 1999


In [4]:
input_loader = DataLoader(input_data, batch_size=batch_size, num_workers=batch_size, drop_last=True)
target_loader1 = DataLoader(target_data1, batch_size=batch_size, num_workers=batch_size, drop_last=True)
target_loader2 = DataLoader(target_data2, batch_size=batch_size, num_workers=batch_size, drop_last=True)
forwarded_target_loader = DataLoader(forwarded_target_data, batch_size=batch_size, num_workers=batch_size, drop_last=True)

print("Prepared DataLoaders")
print("    input_loader:           ", len(input_loader), "batches with size", batch_size)
print("    target_loader1:          ", len(target_loader1), "batches with size", batch_size)
print("    target_loader2:          ", len(target_loader2), "batches with size", batch_size)
print("    forwarded_target_loader:", len(forwarded_target_loader), "batches with size", batch_size)

Prepared DataLoaders
    input_loader:            19 batches with size 100
    target_loader1:           20 batches with size 100
    target_loader2:           20 batches with size 100
    forwarded_target_loader: 19 batches with size 100


/localhome/gawl_ja/micromamba/envs/dl4gam/lib/python3.13/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 100 worker processes in total. Our suggested max number of worker in current system is 22, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
noise_level = 5000
N = 16**2

print("Compute Feasible Appartenance")
distsXX, feasible_appartenance = target_distances_samplingYX_perbatch_cuda(0, input_loader, target_loader1, target_loader2, forwarded_target_loader, p_X=1, p_Y=2, epsilon = (noise_level*N)**(1/2))

print("Convert to Scipy Sparse")
feasible_appartenance = torch_csr_to_scipy(feasible_appartenance)
distsXX = torch_sparse_to_scipy_csr(distsXX)

print("Compute WC Kernelsize")
wc_kersize = kersize_samplingYX(distsXX, feasible_appartenance, p_X = 1)/(N)

print("Compute Avg. Lowerbound")
avg_LB =  avgLB_samplingYX(distsXX, feasible_appartenance, p_X = 1)/(N)

print("Compute Avg. Kernelsize")
avg_kersize = avgkersize_samplingYX(distsXX, feasible_appartenance, p_X = 1)/(N)

print(f'Worst case kernelsize : {wc_kersize} \n Average lower bound : {avg_LB} \n Average kersize : {avg_kersize}')

/localhome/gawl_ja/Dokumente/Kernel/AccuracyBounds/src/accuracy_bounds/inverseproblems/kersize_compute_dataloader.py:561: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  forwarded_target = torch.tensor(forwarded_target, dtype = torch.float32, device = device)
/localhome/gawl_ja/Dokumente/Kernel/AccuracyBounds/src/accuracy_bounds/inverseproblems/kersize_compute_dataloader.py:562: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_data = torch.tensor(input_data, dtype = torch.float32, device = device)
/localhome/gawl_ja/Dokumente/Kernel/AccuracyBounds/src/accuracy_bounds/inverseproblems/kersize_compute_dataloader.py:566: UserWarning: To copy construct from a tensor, it is recommended to use

KeyboardInterrupt: 